In [1]:
import numpy as np
import pandas as pd
import requests
import json
import time

from pandas.io.json import json_normalize

### Notes

#### Extra level of depth problem
<ul>
    <li> May have 4th level of depth, but this level may not be applicable everywhere </li>
    <li> Error catch to ensure this level is captured </li>
    <li> If this level not present for some instances then ensure the subsubsub level is None </li> 
</ul> 

#### What information is important in the dataframe
<ul>
    <li> Location categories - main upwards </li>
    <li> Time of depature</li>
    <li> </li>
</li>

### Main Code

In [2]:
usingWorkLaptop = True

In [3]:
#contains user_id,place_id and datetime of checkin 
if usingWorkLaptop:
    checkins = pd.read_csv("C:\\Users\\hcheena\\OneDrive - KPMG\\Documents\\UserMobilityMining\\gowalla\\gowalla_checkins.csv")
else:
    checkins = pd.read_csv("C:\\Users\\\hasna\\Documents\\Github\\UMM\\gowalla\\gowalla_checkins.csv")

checkins.head()

,userid,placeid,datetime
0,1338,482954,2011-06-23T02:24:22Z
1,1338,580963,2011-06-22T14:23:03Z
2,1338,365256,2011-06-09T23:29:30Z
3,1338,89504,2011-05-22T15:54:30Z
4,1338,1267135,2011-05-21T16:51:13Z


In [4]:
#place_id, longitude, latitude
if usingWorkLaptop:
    placesSubset1 = pd.read_csv("C:\\Users\\hcheena\\OneDrive - KPMG\\Documents\\UserMobilityMining\\gowalla\\gowalla_spots_subset1.csv")
else:
    placesSubset1 = pd.read_csv("C:\\Users\\hasna\\Documents\\Github\\UMM\\gowalla\\gowalla_spots_subset1.csv")

placesSubset1.head()

,id,created_at,lng,lat,photos_count,checkins_count,users_count,radius_meters,highlights_count,items_count,max_items_count,spot_categories
0,8904,2008-12-06T16:28:53Z,-94.607499,39.052318,0,114,21,35,0,10,10,"[{'url': '/categories/89', 'name': 'Craftsman'}]"
1,8932,2008-12-13T02:16:51Z,-97.254356,32.927662,2,67,48,75,0,6,10,"[{'url': '/categories/17', 'name': 'BBQ'}]"
2,8936,2008-12-14T22:08:39Z,-94.591995,39.053318,0,75,46,75,0,10,10,"[{'url': '/categories/103', 'name': 'Theatre'}]"
3,8938,2008-12-15T00:22:49Z,-94.590311,39.052824,38,438,94,50,10,10,10,"[{'url': '/categories/1', 'name': 'Coffee Shop'}]"
4,8947,2008-12-16T23:14:05Z,-122.029631,37.331880,91,3100,1186,200,20,10,10,"[{'url': '/categories/121', 'name': 'Corporate..."


In [38]:
#load the category_structure to relate the places to the main category 

#load json file
if usingWorkLaptop:
    with open("C:\\Users\\hcheena\\OneDrive - KPMG\\Documents\\UserMobilityMining\\gowalla\\gowalla_category_structure.json") as f:
        categoryStructure = json.load(f)['spot_categories']
else:
    with open("C:\\Users\\hasna\\Documents\\Github\\UMM\\gowalla\\gowalla_category_structure.json") as f:
        categoryStructure = json.load(f)['spot_categories']

#flatten json structure
categoryInfo = np.array([])
for firstIndex in range(0, len(categoryStructure)): #main category level
    
    category = categoryStructure[firstIndex]
    categoryName = category['name']
    numberOfSecondLevelCategories = len(category['spot_categories'])
    
    for secondIndex in range(0,numberOfSecondLevelCategories):
        secondLevelCategory = category['spot_categories'][secondIndex]
        secondLevelCategoryName = secondLevelCategory['name']
        numberOfThirdLevelCategories = len(secondLevelCategory['spot_categories'])
        
        #need to make adjustments for an extra level of depth 
        for thirdIndex in range(0,numberOfThirdLevelCategories):
            thirdLevelCategory = secondLevelCategory['spot_categories'][thirdIndex]
            thirdLevelCategoryName = thirdLevelCategory['name']
            
            #form dataframe 
            if categoryInfo.size == 0:
                categoryInfo = np.array([categoryName,secondLevelCategoryName,thirdLevelCategoryName])
            else:
                categoryInfo = np.vstack((categoryInfo,
                                        np.array([categoryName,secondLevelCategoryName,thirdLevelCategoryName])))

#overall category relation table
categoryInfo = pd.DataFrame(categoryInfo, columns=['Main', 'Second', 'Third'])

In [ ]:
#extract the location information 
locationInformation = list()
for rowIndex in placesSubset1.index:
    
    rowInstance = placesSubset1['spot_categories'][rowIndex]
    firstIndex = rowInstance.rfind(":") + 3
    secondIndex = rowInstance.rfind("'")
    
    if len(locationInformation) == 0:
        locationInformation = [rowInstance[firstIndex:secondIndex]]
    else:
        locationInformation.append(rowInstance[firstIndex:secondIndex])
        
#dataframe created for checking purposes
locationDf = pd.DataFrame(locationInformation, columns=['Locations'])
locationDf.head()

In [ ]:
#assemble the dataframe with the location information and the rest of the location data
featuresToExtract = ['id','created_at', 'lng', 'lat']
locationFeatureDf = placesSubset1[featuresToExtract]

relatedMainCategory = list() 
relatedSecondCategory = list()
relatedThirdCategory = list()

#add to locationDf the main, sub categories
#do do this locate whether the given location is in sub or subsub and then paste the associated main category value threre to create the entire dataframe

#check search for location column wise 
for index in range(0,locationDf.shape[0]):
    
    if index % 10000 == 0:
        print(index)
    
    searchLocation = locationDf['Locations'][index]
    
    #determine where in either mask the true value lies
    thirdMask = np.equal(searchLocation, categoryInfo['Third'])
#     secondMask = np.equal(searchLocation, categoryInfo['Second'])
#     searchMask = np.logical_or(secondMask, thirdMask)
    
    #subset and then append to list
    mainCategory = categoryInfo['Main'][searchMask]
    secondCategory = categoryInfo['Second'][searchMask]
    thirdCategory = categoryInfo['Third'][searchMask]

    
    relatedMainCategory.append(mainCategory)
    relatedSecondCategory.append(secondCategory)
    relatedThirdCategory.append(thirdCategory)

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
490000
500000
510000
520000
530000
540000
550000
560000
570000
580000
590000
600000
610000
620000
630000
640000
650000
660000


In [ ]:
#assemble df
locationFeatureDf['relatedMainCategory'] = relatedMainCategory
locationFeatureDf['relatedSecondaryCategory'] = relatedSecondCategory
locationFeatureDf['relatedThirdCategory'] = relatedThirdCategory

In [ ]:
#save locationFeatureDf
locationFeatureDf.to_csv("LocationFeatureData.csv")

In [ ]:
#create date-time feature
dateTime = checkins['datetime']
